# 神经网络 后向传播实现版本
实现各个函数的后向传播流程，最后生成一个完整的神经网络

In [12]:
import numpy as np
from collections import OrderedDict

In [9]:
def softmax(z):
    max_value = np.max(z)
    exp_z = np.exp(z - max_value)
    sum_max_z = np.sum(exp_z)
    y = exp_z/sum_max_z
    return y

In [10]:
def cross_entropy_error(y, t):
    delta = 1e-7
    if y.ndim ==1:
        y = y.reshape(1, y.size)
        t = t.reshape(1, t.size)
        
    batch_size = y.shape[0]
    return -np.sum(t*np.log(y+delta))/batch_size

In [2]:
# reLU函数的正向、反向传播
class reLU:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x<= 0)
        out = x.copy()
        out[self.mask] =0
        return out
    
    # reLU 函数的 偏导数如图所示
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx
        

In [4]:
# sigmoid 层的正向、反向传播
class sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1/(1+np.exp(-x))
        self.out = out
    
        return out
    
    def backward(self, dout):
        dx = dout*(1.0 - self.out) *self.out
        return dx
        

In [5]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dw = None
        self.db = None
    
    def forward(self, x, b):
        self.x = x
        out = x@self.W + b
        
        return out
    
    def backward(self, dout):
        dx = dout @ self.W.T
        self.dw = self.x.T @ dout
        self.db = np.sum(dout, axis=0)
        
        return dx

In [11]:
class softmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t)/batch_size
        return dx

In [ ]:
# 拼接整个网络模块
class twoLayerNetwork:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['reLU'] = reLU()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.last_layer = softmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)
    
    def gradient(self, x, t):
        self.loss(x, t)
        
        dout = 1
        dout = self.last_layer.backward(dout)
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        grads = {}
        grads['W1'] = self.layers['Affine1'].dw
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dw
        grads['b2'] = self.layers['Affine2'].db
        
        return grads